In [18]:
%matplotlib inline  
import pandas as pd
import numpy as np
import os, sys, time
import yaml
from sklearn.metrics import matthews_corrcoef

sys.path.append('/home/ymm/kaggle/xgboost_hyperopt')
import utils
from utils.bosch_functions import load_processed_bosch_data
from utils.models import CombinedModel, ExtraTreeModel
from utils.validation_tools import score_MCC, create_validation_index, cross_validate_model
from utils.wrapped_xgboost import xgboost_classifier

In [4]:
'''
project_path = '/mnt/home/ymm/kaggle/compete/current'
data_path = '/mnt/home/ymm/kaggle/bosch_processed_data'

old way to import data
yaml_file = 'bosch_processed_data_dict.yml'
with open(os.path.join(project_path, yaml_file), 'r') as yml_stream:
    data_dict = yaml.load(yml_stream)

data_index = '0'
data_file = os.path.join(data_path, data_dict[data_index]['train_file'])
print 'loading data from ', data_file
train = pd.read_csv(data_file, index_col='Id')
'''

"\nproject_path = '/mnt/home/ymm/kaggle/compete/current'\ndata_path = '/mnt/home/ymm/kaggle/bosch_processed_data'\n\nold way to import data\nyaml_file = 'bosch_processed_data_dict.yml'\nwith open(os.path.join(project_path, yaml_file), 'r') as yml_stream:\n    data_dict = yaml.load(yml_stream)\n\ndata_index = '0'\ndata_file = os.path.join(data_path, data_dict[data_index]['train_file'])\nprint 'loading data from ', data_file\ntrain = pd.read_csv(data_file, index_col='Id')\n"

In [30]:
## load training data
#project_path = '/mnt/home/ymm/kaggle/compete/current'
#data_path = '/mnt/home/ymm/kaggle/bosch_data/bosch_complete_processed_data'
#data_yaml_file = 'bosch_processed_data_dict.yml'

project_path = '/home/ymm/kaggle/compete/current/model_6_bins_data'
data_path = '/home/ymm/kaggle/bosch_data/bosch_complete_processed_6_bins_data'
data_yaml_file = 'complete_subset_data_6_bins_dict.yml'

dep_var_name = 'Response'
data_index= '0'
train = load_processed_bosch_data(data_path, project_path, data_yaml_file, data_index=data_index)
train_index, valid_index = create_validation_index(train, 0.5, dep_var_name, True)  
valid_data = train.ix[valid_index]
tmp_train  = train.ix[train_index]

loading bosch data from  /home/ymm/kaggle/bosch_data/bosch_complete_processed_6_bins_data/processed_totBins_6_bin_0_train.csv


In [6]:
#reload(utils.wrapped_xgboost)
#dir(utils.wrapped_xgboost)
#reload(utils.validation_tools)

In [147]:
'''
from utils.validation_tools import create_validation_index
## obtein the index for train and validation data
train_index, valid_index = create_validation_index(train, dep_var_name='Response', valid_frac=0.3)
'''

"\nfrom utils.validation_tools import create_validation_index\n## obtein the index for train and validation data\ntrain_index, valid_index = create_validation_index(train, dep_var_name='Response', valid_frac=0.3)\n"

### xgboost for imbalanced data

#### data preparation
    a. use the larger dataset, first bin out of 6-bins data
    b. a fully processed data, missing values are already filled

#### model benchmark
    params["eta"]                      = 0.01
    params["subsample"]                = 0.8
    params["colsample_bytree"]         = 0.8
    params["num_round"]                = 500

#### approaches
    1. Use param 'scale_pos_weight', as the ratio of negative samples to positive ones
     
     a. 500 num_round, 3-folds results: 
     [0.17132387528893958, 0.18898085550009494, 0.15096433310166174]
     b. 1000 num_round, 3-folds results:
     [0.1502914863368823, 0.16363650723447276, 0.17630868136728392]
     c. 1500 num_round, 3-fold results:
     
    2. vanilla xgboost without any additional modification.
     the gradient increase rate is significantly slower compared to the case#1, so 
     1000 num_rouns are used instead of 500
     
     3-folds results: 
    [0.16291091970811675, 0.18898085550009464, 0.16363650723447307]

    2. Assign proper weights to samples
        a. use 500 num_round, the train-AUC does not reach 0.9999
        3-folds results:
        [0.16711739749852803, 0.17630868136728373, 0.18898085550009505]
        b. use 1000 num_round
        [0.12925909738482472, 0.16786056527874313, 0.17630868136728384]


In [31]:
scale_pos_weight = 1. * np.sum(train[dep_var_name].values == 0) / np.sum(train[dep_var_name].values == 1)
print scale_pos_weight

In [52]:
dep_var_name = 'Response'

params = {}
params["eta"]                      = 0.01
params["subsample"]                = 0.8
params["colsample_bytree"]         = 0.8
params["num_round"]                = 1500
params["max_depth"]                = 5
params["gamma"]                    = 0
params["metrics"]                  = 'auc'
params['eval_metric']              = 'auc'
params["seed"]                     = 100
params["val"]                      = False
params["early_stopping_ratio"]     = 0.2
## important param for highly imbalanced data
params['scale_pos_weight']         = scale_pos_weight

In [53]:
xgb_clf = xgboost_classifier(label_name = dep_var_name, params = params)

In [54]:
results = xgb_clf.cross_validate_fit(score_MCC, train, n_folds=3)

label name: Response
train shape: (134779, 2269)



####################
 train the xgboost without early stopping
####################


[0]	train-auc:0.715453
[1]	train-auc:0.76995
[2]	train-auc:0.788205
[3]	train-auc:0.795868
[4]	train-auc:0.798008
[5]	train-auc:0.803253
[6]	train-auc:0.810902
[7]	train-auc:0.81001
[8]	train-auc:0.814529
[9]	train-auc:0.819272
[10]	train-auc:0.818117
[11]	train-auc:0.819862
[12]	train-auc:0.82052
[13]	train-auc:0.822219
[14]	train-auc:0.822856
[15]	train-auc:0.823783
[16]	train-auc:0.827519
[17]	train-auc:0.828832
[18]	train-auc:0.828732
[19]	train-auc:0.827776
[20]	train-auc:0.830451
[21]	train-auc:0.83171
[22]	train-auc:0.836345
[23]	train-auc:0.836523
[24]	train-auc:0.836299
[25]	train-auc:0.838943
[26]	train-auc:0.838375
[27]	train-auc:0.838946
[28]	train-auc:0.838013
[29]	train-auc:0.838376
[30]	train-auc:0.839573
[31]	train-auc:0.840562
[32]	train-auc:0.841729
[33]	train-auc:0.842692
[34]	train-auc:0.843974
[35]	train-auc:0.846855
[36]	train-auc:0.847997
[37]	train-auc:0.84982
[38]	train-auc:0.851221
[39]	train-auc:0.851028
[40]	train-auc:0.85107
[41]	train-auc:0.850641
[42]	tra


####################
 train the xgboost without early stopping
####################


[0]	train-auc:0.753701
[1]	train-auc:0.776472
[2]	train-auc:0.801519
[3]	train-auc:0.808644
[4]	train-auc:0.810749
[5]	train-auc:0.814334
[6]	train-auc:0.818377
[7]	train-auc:0.825349
[8]	train-auc:0.827247
[9]	train-auc:0.828269
[10]	train-auc:0.824853
[11]	train-auc:0.828862
[12]	train-auc:0.829893
[13]	train-auc:0.834905
[14]	train-auc:0.835807
[15]	train-auc:0.837909
[16]	train-auc:0.838942
[17]	train-auc:0.841371
[18]	train-auc:0.842385
[19]	train-auc:0.844508
[20]	train-auc:0.846562
[21]	train-auc:0.846598
[22]	train-auc:0.847678
[23]	train-auc:0.849296
[24]	train-auc:0.850977
[25]	train-auc:0.851012
[26]	train-auc:0.850836
[27]	train-auc:0.852188
[28]	train-auc:0.852384
[29]	train-auc:0.85251
[30]	train-auc:0.852354
[31]	train-auc:0.853869
[32]	train-auc:0.854558
[33]	train-auc:0.854746
[34]	train-auc:0.855775
[35]	train-auc:0.856416
[36]	train-auc:0.85748
[37]	train-auc:0.85839
[38]	train-auc:0.859228
[39]	train-auc:0.860546
[40]	train-auc:0.862432
[41]	train-auc:0.862905
[42]	


####################
 train the xgboost without early stopping
####################


[0]	train-auc:0.756533
[1]	train-auc:0.792556
[2]	train-auc:0.804419
[3]	train-auc:0.813206
[4]	train-auc:0.821107
[5]	train-auc:0.828204
[6]	train-auc:0.828149
[7]	train-auc:0.829987
[8]	train-auc:0.831257
[9]	train-auc:0.831338
[10]	train-auc:0.830851
[11]	train-auc:0.834859
[12]	train-auc:0.834418
[13]	train-auc:0.835396
[14]	train-auc:0.836042
[15]	train-auc:0.834733
[16]	train-auc:0.836409
[17]	train-auc:0.83908
[18]	train-auc:0.839672
[19]	train-auc:0.839659
[20]	train-auc:0.839705
[21]	train-auc:0.840383
[22]	train-auc:0.841407
[23]	train-auc:0.842795
[24]	train-auc:0.842678
[25]	train-auc:0.844057
[26]	train-auc:0.843372
[27]	train-auc:0.842826
[28]	train-auc:0.844621
[29]	train-auc:0.846456
[30]	train-auc:0.848213
[31]	train-auc:0.848191
[32]	train-auc:0.850125
[33]	train-auc:0.851215
[34]	train-auc:0.852257
[35]	train-auc:0.852272
[36]	train-auc:0.853674
[37]	train-auc:0.854519
[38]	train-auc:0.855511
[39]	train-auc:0.85587
[40]	train-auc:0.857378
[41]	train-auc:0.858756
[42]

In [51]:
print results

[0.1502914863368823, 0.16363650723447276, 0.17630868136728392]


### single xgb model

In [141]:
reload(utils.validation_tools)
dir(utils.validation_tools)

['StratifiedKFold',
 '__builtins__',
 '__doc__',
 '__file__',
 '__name__',
 '__package__',
 'create_validation_index',
 'cross_validate_model',
 'matthews_corrcoef',
 'np',
 'os',
 'pd',
 'score_MCC',
 'shuffle',
 'sys',
 'time']

In [38]:
xgb_train_data = train.ix[train_index]
xgb_test_data = train.ix[valid_index]

In [51]:
xgb_clf = xgboost_classifier(xgb_train_data, dep_var_name, params = params)
xgb_clf.fit()


####################
 train the xgboost without early stopping
####################


[0]	train-auc:0.505492
[1]	train-auc:0.505492
[2]	train-auc:0.516412
[3]	train-auc:0.516412
[4]	train-auc:0.527302
[5]	train-auc:0.527272
[6]	train-auc:0.591767
[7]	train-auc:0.591868
[8]	train-auc:0.59186
[9]	train-auc:0.591876
[10]	train-auc:0.591876
[11]	train-auc:0.591877
[12]	train-auc:0.591864
[13]	train-auc:0.591861
[14]	train-auc:0.597336
[15]	train-auc:0.597341
[16]	train-auc:0.597227
[17]	train-auc:0.597221
[18]	train-auc:0.597218
[19]	train-auc:0.59719
[20]	train-auc:0.59719
[21]	train-auc:0.597188
[22]	train-auc:0.597188
[23]	train-auc:0.597193
[24]	train-auc:0.597196
[25]	train-auc:0.5972
[26]	train-auc:0.602639
[27]	train-auc:0.602647
[28]	train-auc:0.602645
[29]	train-auc:0.602644
[30]	train-auc:0.602629
[31]	train-auc:0.602621
[32]	train-auc:0.602626
[33]	train-auc:0.602624
[34]	train-auc:0.602626
[35]	train-auc:0.602627
[36]	train-auc:0.602626
[37]	train-auc:0.602626
[38]	train-auc:0.602626
[39]	train-auc:0.602597
[40]	train-auc:0.602596
[41]	train-auc:0.602598
[42]	tr

In [41]:
pred_res = xgb_clf.predict(xgb_test_data)
validation_tools.score_MCC(xgb_test_data['Response'].values, pred_res)

In [43]:
sys.path.append('/home/ymm/kaggle/xgboost_hyperopt')
import utils.validation_tools as validation_tools

In [47]:
reload(validation_tools)
dir(validation_tools)

<module 'utils.validation_tools' from '/home/ymm/kaggle/xgboost_hyperopt/utils/validation_tools.py'>